In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# 0. Precurser Info
## 0.a Variable Descriptions
* survival	Survival	0 = No, 1 = Yes
* pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
* sex	male, female
* Age	Age in years
* sibsp	# of siblings / spouses aboard the Titanic
* parch	# of parents / children aboard the Titanic
* ticket	Ticket number
* fare	Passenger fare
* cabin	Cabin number
* embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

## 0.b Additional Import Statements

In [2]:
from sklearn.ensemble import RandomForestClassifier #in order to use a random forest classification tree

# 1. Import Training and Testing Data

In [3]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
train_data.head() #just figured out that excluding the print statement shows a more legible than using the print() output

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 2. Example Case: Outcome Model based on Gender

In [4]:
women = train_data.loc[train_data.Sex == 'female']['Survived']
rate_women = np.round(100*sum(women)/len(women),3)

print(f'{rate_women}% of women survived the titanic incident')

men = train_data.loc[train_data.Sex == 'male']['Survived']
rate_men = np.round(100*sum(men)/len(men),3)

print(f'{rate_men}% of men survived the titanic incident')

74.204% of women survived the titanic incident
18.891% of men survived the titanic incident


# 3. Example Case: Outcome Model based on Pclass, Sex, SibSp, Parch

In [5]:
y = train_data['Survived']

features = ['Pclass', 'Sex', 'SibSp', 'Parch']
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators = 100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print('submission successfully saved... but to where... found it =)')

submission successfully saved... but to where... found it =)


# 4. Case: Outcome Model based on Pclass, Sex, SibSp, Parch, Embarked

In [6]:
#Quick check for variables that may be missing data in one form or another (Age, Cabin and Embarked each may require soem adjusting to use)

#Train Data NaN values / missing values
print('Training Data Status')
print(train_data.isna().sum())
print()

#Test Data NaN values / missing values
print('Test Data Status')
print(test_data.isna().sum())

Training Data Status
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Test Data Status
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [7]:
train_data_2 = train_data.dropna(subset=['Embarked'], how='all')
print(train_data_2.info())
y = train_data_2['Survived']

features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']
X = pd.get_dummies(train_data_2[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=1) #Traying at estimators=200 for next submission
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print('done... again')

<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Name         889 non-null    object 
 4   Sex          889 non-null    object 
 5   Age          712 non-null    float64
 6   SibSp        889 non-null    int64  
 7   Parch        889 non-null    int64  
 8   Ticket       889 non-null    object 
 9   Fare         889 non-null    float64
 10  Cabin        202 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 90.3+ KB
None
done... again
